In [1]:
from flask import Flask, render_template, request, jsonify
import os
from werkzeug.utils import secure_filename
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import numpy as np
import requests

app = Flask(__name__)
app.config['SECRET_KEY'] = 'brain-tumor-secret-key'
app.config['UPLOAD_FOLDER'] = 'static/uploads'
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB

# Create directories
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

# Allowed extensions
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'bmp'}

class BrainTumorPredictor:
    def __init__(self, model_path=None):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")
        
        # Try multiple possible model paths
        possible_paths = [
            r'C:\Users\Taif\Desktop\brain api\models\reliable_model.pth',
            'models/reliable_model.pth',
            './models/reliable_model.pth',
            'reliable_model.pth',
            '../models/reliable_model.pth'
        ]
        
        # Use the first path that exists, or None if none exist
        self.model_path = None
        for path in possible_paths:
            if os.path.exists(path):
                self.model_path = path
                print(f"✅ Found model at: {path}")
                break
        
        if not self.model_path:
            print("❌ Model file not found in any of the expected locations")
            print("🔄 Using demonstration mode with mock predictions")
                
        self.class_names = ['glioma', 'meningioma', 'pituitary', 'notumor']
        self.model = None
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])
        self.load_model()
    
    def load_model(self):
        """Load the trained model"""
        try:
            if self.model_path and os.path.exists(self.model_path):
                print(f"📥 Loading brain tumor model from {self.model_path}...")
                
                # Load checkpoint
                checkpoint = torch.load(self.model_path, map_location=self.device)
                
                # Create model architecture - try different approaches
                if 'model_state_dict' in checkpoint:
                    # Model was saved as state dict
                    self.model = models.resnet18(pretrained=False)
                    self.model.fc = nn.Linear(self.model.fc.in_features, 4)
                    self.model.load_state_dict(checkpoint['model_state_dict'])
                elif 'state_dict' in checkpoint:
                    # Alternative state dict key
                    self.model = models.resnet18(pretrained=False)
                    self.model.fc = nn.Linear(self.model.fc.in_features, 4)
                    self.model.load_state_dict(checkpoint['state_dict'])
                else:
                    # Model was saved directly
                    self.model = checkpoint
                
                self.model.to(self.device)
                self.model.eval()
                
                print("✅ Model loaded successfully!")
                
                # Print model info
                if hasattr(checkpoint, 'get'):
                    if 'val_acc' in checkpoint:
                        print(f"🏆 Original validation accuracy: {checkpoint['val_acc']:.2f}%")
                
                # Test model with random input
                self.test_model()
                
            else:
                print("🔄 Setting up mock model for demonstration...")
                self.setup_mock_model()
                
        except Exception as e:
            print(f"❌ Error loading model: {e}")
            print("🔄 Setting up mock model for demonstration...")
            self.setup_mock_model()
    
    def test_model(self):
        """Test if model works with random input"""
        try:
            test_input = torch.randn(1, 3, 224, 224).to(self.device)
            with torch.no_grad():
                output = self.model(test_input)
            print(f"✅ Model test passed! Output shape: {output.shape}")
            return True
        except Exception as e:
            print(f"❌ Model test failed: {e}")
            return False
    
    def setup_mock_model(self):
        """Setup mock model for demonstration"""
        try:
            self.model = models.resnet18(pretrained=True)
            self.model.fc = nn.Linear(self.model.fc.in_features, 4)
            self.model.to(self.device)
            self.model.eval()
            print("✅ Mock model setup completed")
        except Exception as e:
            print(f"❌ Error setting up mock model: {e}")
    
    def predict(self, image_path):
        """Predict brain tumor type from image"""
        try:
            # Load and preprocess image
            image = Image.open(image_path).convert('RGB')
            
            # Apply transformations
            image_tensor = self.transform(image).unsqueeze(0).to(self.device)
            
            # Check if model is available
            if self.model is None:
                print("⚠️ Model not available, using mock prediction")
                return self.mock_prediction()
            
            # Make prediction
            with torch.no_grad():
                outputs = self.model(image_tensor)
                probabilities = torch.softmax(outputs, dim=1)
                confidence, predicted = torch.max(probabilities, 1)
            
            predicted_class = self.class_names[predicted.item()]
            confidence = confidence.item()
            all_probabilities = probabilities.cpu().numpy()[0]
            
            print(f"🎯 Prediction: {predicted_class}, Confidence: {confidence:.4f}")
            
            return {
                'success': True,
                'predicted_class': predicted_class,
                'confidence': confidence,
                'all_probabilities': all_probabilities.tolist(),
                'class_names': self.class_names,
                'mock': False
            }
            
        except Exception as e:
            print(f"❌ Prediction error: {e}")
            # Return mock prediction for demonstration
            return self.mock_prediction()
    
    def mock_prediction(self):
        """Generate mock prediction for demonstration"""
        try:
            import random
            # Create more realistic probabilities
            base_probs = [random.uniform(0.1, 0.3) for _ in range(4)]
            # Make one class significantly higher
            dominant_class = random.randint(0, 3)
            base_probs[dominant_class] += random.uniform(0.4, 0.6)
            
            # Normalize
            total = sum(base_probs)
            probabilities = [p/total for p in base_probs]
            
            max_idx = probabilities.index(max(probabilities))
            predicted_class = self.class_names[max_idx]
            confidence = probabilities[max_idx]
            
            print(f"🤖 Mock Prediction: {predicted_class}, Confidence: {confidence:.4f}")
            
            return {
                'success': True,
                'predicted_class': predicted_class,
                'confidence': confidence,
                'all_probabilities': probabilities,
                'class_names': self.class_names,
                'mock': True  # Flag to indicate this is a mock prediction
            }
        except Exception as e:
            print(f"❌ Mock prediction error: {e}")
            return {
                'success': False,
                'error': 'Prediction failed'
            }

# Initialize predictor ONCE at startup
print("🚀 Initializing Brain Tumor Predictor...")
predictor = BrainTumorPredictor()
print("✅ Predictor initialized!")

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        if 'file' not in request.files:
            return jsonify({'success': False, 'error': 'No file uploaded'})
        
        file = request.files['file']
        
        if file.filename == '':
            return jsonify({'success': False, 'error': 'No file selected'})
        
        if file and allowed_file(file.filename):
            # Save uploaded file
            filename = secure_filename(file.filename)
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(filepath)
            
            print(f"📁 Processing file: {filename}")
            
            # Make prediction
            result = predictor.predict(filepath)
            
            if result['success']:
                # Fix percentage formatting if needed
                confidence = result['confidence']
                if confidence > 1:  # If confidence is > 1, it's probably not normalized
                    confidence = confidence / 100
                
                response_data = {
                    'success': True,
                    'prediction': result['predicted_class'],
                    'confidence': f"{confidence:.2%}",
                    'probabilities': [
                        {'class': cls, 'probability': f"{prob:.2%}"}
                        for cls, prob in zip(result['class_names'], result['all_probabilities'])
                    ],
                    'image_url': f'/static/uploads/{filename}'
                }
                
                if result.get('mock'):
                    response_data['note'] = 'This is a demonstration using mock data'
                    
                return jsonify(response_data)
            else:
                return jsonify({'success': False, 'error': result.get('error', 'Prediction failed')})
        else:
            return jsonify({'success': False, 'error': 'Invalid file type'})
            
    except Exception as e:
        print(f"❌ Route error: {e}")
        return jsonify({'success': False, 'error': str(e)})

@app.route('/test-model')
def test_model():
    """Test endpoint to check model status"""
    try:
        test_result = predictor.test_model() if hasattr(predictor, 'test_model') else False
        return jsonify({
            'model_loaded': predictor.model is not None,
            'model_path': predictor.model_path,
            'device': str(predictor.device),
            'test_passed': test_result
        })
    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    print("🚀 Starting Brain Tumor Classification Web App...")
    print("🌐 Open http://localhost:8080 in your browser")
    print("📁 Upload folder:", app.config['UPLOAD_FOLDER'])
    print("📁 Current directory:", os.getcwd())
    print("📁 Files in current directory:", os.listdir('.'))
    
    # Check if models directory exists
    if os.path.exists('models'):
        print("📁 Files in models directory:", os.listdir('models'))
    else:
        print("❌ Models directory not found")
    
    app.run(debug=True, host='127.0.0.1', port=8080, use_reloader=False)

🚀 Initializing Brain Tumor Predictor...
Using device: cuda
✅ Found model at: C:\Users\Taif\Desktop\brain api\models\reliable_model.pth
📥 Loading brain tumor model from C:\Users\Taif\Desktop\brain api\models\reliable_model.pth...


c:\Users\Taif\Desktop\brain api\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Taif\Desktop\brain api\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


✅ Model loaded successfully!
🏆 Original validation accuracy: 94.08%
✅ Model test passed! Output shape: torch.Size([1, 4])
✅ Predictor initialized!
🚀 Starting Brain Tumor Classification Web App...
🌐 Open http://localhost:8080 in your browser
📁 Upload folder: static/uploads
📁 Current directory: c:\Users\Taif\Desktop\brain api
📁 Files in current directory: ['.venv', 'app.ipynb', 'models', 'pytorch_env', 'requirements.txt', 'static', 'templates']
📁 Files in models directory: ['reliable_model.pth']
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit
127.0.0.1 - - [26/Oct/2025 10:15:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2025 10:15:56] "POST /predict HTTP/1.1" 200 -


📁 Processing file: image15.jpg
🎯 Prediction: pituitary, Confidence: 0.9865


127.0.0.1 - - [26/Oct/2025 10:15:57] "GET /static/uploads/image15.jpg HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2025 10:16:01] "GET /static/uploads/image15.jpg HTTP/1.1" 304 -
